Image retrieval based on the Hindi to English translated captions.

This model only supports English captions.

# **Loading the dataset**

In [ ]:
import os
from google.colab import drive
from PIL import Image
from numpy import asarray
import random
import numpy as np

In [ ]:
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.4 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalled tensorboard-2.14.1
  Attempting uninstall: tensorflow
    Found existing install

In [ ]:
from tensorflow import keras
from keras import layers
from keras import applications
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
seed = 21
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
caption_path = '/content/drive/MyDrive/CSCI567/Flickr8k_text/Flickr-Hindi2Eng-translated.txt'
f = open(caption_path)
lines = f.readlines()
captions = [] # stores entries in format - label(image name), caption
for line in lines:
  parts = line.split("	")
  captions.append([parts[0].split("#")[0], parts[1]])

captions = np.array(captions)

In [ ]:
def column(arr, i):
    return [row[i] for row in arr]

In [ ]:
# creating a list of the 500 unique images that the captions txt file has
print(len(column(captions,0)))
unique_images = set(column(captions,0))
print(len(unique_images))

2500
500


In [ ]:
image_directory = '/content/drive/MyDrive/CSCI567/Flickr8k_Dataset/Flicker8k_Dataset'
image_paths_temp = [os.path.join(image_directory, filename) for filename in unique_images]

In [ ]:
print(len(image_paths_temp))

500


In [ ]:
# image paths which correspond to actual images in the dataset
image_paths = []
for path in image_paths_temp:
  if os.path.isfile(path):
    image_paths.append(path)

In [ ]:
print(len(image_paths))

229


In [ ]:
images = []

for path in image_paths:
  image = Image.open(path).resize((64,64))
  data = asarray(image) # might need to reshape
  image_label = path.split(image_directory+'/')[1]
  images.append([image_label,data])
  #print(image_label, data)

In [ ]:
# TEST-TRAIN SPLIT
#print(captions)
np.random.shuffle(captions)
#print(captions)
split = int(len(captions)*0.8)
training_captions, test_captions = captions[:split,:], captions[split:,:]
print(len(training_captions), len(test_captions))

2000 500


**Creating triplets**

In [ ]:
def triplet_generator(images, captions, num_triplets_per_caption):
    imgs = []
    positives = []
    negatives = []
    # loop through the captions, each caption and it's corresponding image will be stored with one random negative caption, creating a triplet
    for caption in captions:
        positive = caption[1]
        label = caption[0]
        image = None

        for img in images:
            if img[0] == label:
                image = np.array(img[1])

        if image is None:
            continue

        for i in range(num_triplets_per_caption):
          while True:
            chosen_caption = random.choice(captions)
            l = chosen_caption[0]
            if l != label:
                negative = chosen_caption[1]
                break
          # triplets.append([image, positive, negative])
          imgs.append(image)
          positives.append(positive)
          negatives.append(negative)

    print(f"Dataset size: {len(imgs)}")
    dummy_labels = np.array([0] * len(imgs))
    return np.array(imgs, dtype='float32'), np.array(positives), np.array(negatives), dummy_labels

In [ ]:
imgs, positives, negatives, dummy_labels = triplet_generator(images, training_captions, 3)

Dataset size: 2751


In [ ]:
print(np.shape(positives))

(2751,)


In [ ]:
print(positives[0])

Two boys are passing through the road walking the bicycle 



# **Model**

In [ ]:
# TRIPLET LOSS- referred to https://github.com/prabhnoor0212/Kaggle-Recognizing-faces-in-the-wild/blob/master/kinship_triplet_loss.ipynb

def triplet_loss(y_pred, alpha = 0.5):
    # print(y_true.shape, y_pred.shape)
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    loss = tf.reduce_mean(tf.maximum(basic_loss, 0.0))

    return loss

In [ ]:
# Image base
img_base = applications.ResNet50(weights = 'imagenet', include_top=False, input_shape= (64,64,3), pooling='avg')

# Text base

tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

preprocessor = hub.load(tfhub_handle_preprocess)
tokenize = hub.KerasLayer(preprocessor.tokenize)
seq_length = 8
bert_pack_inputs = hub.KerasLayer(
    preprocessor.bert_pack_inputs,
    arguments=dict(seq_length=seq_length))

bert_preprocess_model = layers.Lambda(lambda x: bert_pack_inputs([tokenize(x)]))
bert_model = hub.KerasLayer(tfhub_handle_encoder, name="text_bert_output")

94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
# MAIN MODEL ARCHITECTURE

img_inputs = layers.Input(shape=(64,64,3), dtype="float32", name="input_image")
positive_text_inputs = layers.Input(shape=(), dtype=tf.string, name="positive_input_captions")
negative_text_inputs = layers.Input(shape=(), dtype=tf.string, name="negative_input_captions")

# positive text embedding
text_preprocessed_p = bert_preprocess_model(positive_text_inputs)
bert_embedded_p = bert_model(text_preprocessed_p)
embedded_p = tf.reduce_mean(bert_embedded_p['sequence_output'], axis=1)

# negative text embedding
text_preprocessed_n = bert_preprocess_model(negative_text_inputs)
bert_embedded_n = bert_model(text_preprocessed_n)
embedded_n = tf.reduce_mean(bert_embedded_n['sequence_output'], axis=1)

# image embedding
image_embedding = img_base(img_inputs)

n_hidden = 12
l2_norm_layer = layers.Lambda(lambda x: tf.math.l2_normalize(x))

text_dense_layer = keras.Sequential([
  layers.Dense(n_hidden, activation="relu", name="text_dense_1"),
  layers.Dense(n_hidden, name="text_dense_2"),
  l2_norm_layer,
])

image_dense_layer = keras.Sequential([
  layers.Dense(n_hidden, activation="relu", name="image_dense_1"),
  layers.Dense(n_hidden, name="image_dense_2"),
  l2_norm_layer,
])


positive_emb = text_dense_layer(embedded_p)

negative_emb = text_dense_layer(embedded_n)

image_emb = image_dense_layer(image_embedding)

In [ ]:
MODEL = keras.Model([img_inputs, positive_text_inputs, negative_text_inputs], [image_emb, positive_emb, negative_emb])

# MODEL.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=0.0001),loss=triplet_loss)

print(MODEL.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 positive_input_captions (I  [(None,)]                    0         []                            
 nputLayer)                                                                                       
                                                                                                  
 negative_input_captions (I  [(None,)]                    0         []                            
 nputLayer)                                                                                       
                                                                                                  
 lambda (Lambda)             {'input_word_ids': (None,    0         ['positive_input_captions[0][0
                             8),                                    ]',                       

In [ ]:
epochs = 8
optimizer=tf.keras.optimizers.AdamW(learning_rate=0.0002)
dataset = tf.data.Dataset.from_tensor_slices((imgs, positives, negatives)).shuffle(buffer_size=256).batch(64)
for epoch in range(epochs):
    print(f"\nStart of epoch {epoch}")
    # start_time = time.time()

    for step, (imgs_batch, positives_batch, negatives_batch) in enumerate(dataset):
        with tf.GradientTape() as tape:
            output = MODEL([imgs_batch, positives_batch, negatives_batch], training=True)
            # print(f"output shape - {output.shape}")
            loss = triplet_loss(output)
        grads = tape.gradient(loss, MODEL.trainable_weights)
        optimizer.apply_gradients(zip(grads, MODEL.trainable_weights))

        print(f"Training loss (for 1 batch) at step {step}: {float(loss):.4f}")


Start of epoch 0
Training loss (for 1 batch) at step 0: 0.4999
Training loss (for 1 batch) at step 1: 0.4987
Training loss (for 1 batch) at step 2: 0.4982
Training loss (for 1 batch) at step 3: 0.4983
Training loss (for 1 batch) at step 4: 0.4968
Training loss (for 1 batch) at step 5: 0.4976
Training loss (for 1 batch) at step 6: 0.4972
Training loss (for 1 batch) at step 7: 0.4964
Training loss (for 1 batch) at step 8: 0.4980
Training loss (for 1 batch) at step 9: 0.4983
Training loss (for 1 batch) at step 10: 0.4963
Training loss (for 1 batch) at step 11: 0.4972
Training loss (for 1 batch) at step 12: 0.4975
Training loss (for 1 batch) at step 13: 0.4971
Training loss (for 1 batch) at step 14: 0.4986
Training loss (for 1 batch) at step 15: 0.4967
Training loss (for 1 batch) at step 16: 0.4978
Training loss (for 1 batch) at step 17: 0.4963
Training loss (for 1 batch) at step 18: 0.4957
Training loss (for 1 batch) at step 19: 0.4974
Training loss (for 1 batch) at step 20: 0.4974
Train

In [ ]:
#MODEL.save("5epochs-6652.keras")

# **Evaluation**

In [ ]:
eval_images = set(column(test_captions,0))
print(len(eval_images))

eval_captions = test_captions

332


In [ ]:
retrieval_set = []
valid_queries = []
valid_images = []

# given all the test image file names, set aside those which actually have a corresponding image
for i in eval_images:
  for img in images:
      if img[0] == i:
          valid_images.append(img[0]) # list of valid labels
          retrieval_set.append(img[1]) # list of images
print(len(retrieval_set))

# list of those captions which have a corresponding image in valid_images
for cap in eval_captions:
  if cap[0] in valid_images:
    valid_queries.append(cap)

retrieval_set = np.array(retrieval_set, dtype='float32')

159


In [ ]:
print(len(valid_queries))

### **Image Embedding**

In [ ]:
image_embeddings = []
for img in retrieval_set:
  #print(img)
  input = img.reshape(-1, 64, 64, 3)
  dummy_caps = np.array(['dummy'])
  output = MODEL([input, dummy_caps, dummy_caps], training=False)
  image_embeddings.append(output[0])

In [ ]:
print(np.array(image_embeddings).shape)
print(image_embeddings[2])

(159, 1, 12)
tf.Tensor(
[[ 0.27174082  0.24554104 -0.37611887 -0.0767191   0.29210466  0.15879098
   0.29993552  0.04522039  0.53058577  0.44372845 -0.00884548 -0.19358432]], shape=(1, 12), dtype=float32)


### **Text Embedding, Top K retrieved Images**

For all valid test queries

In [ ]:
k = 50
total_queries = len(valid_queries)
correct_output = 0

for q in valid_queries:
  label = q[0]
  query = np.array([q[1]])
  distances = []

  # feed the query into the model with any image - we only care about the text embedding generated
  output = MODEL([imgs[0].reshape(-1, 64, 64, 3), query, query], training=False)
  query_embedding = output[1]

  for img_emb in image_embeddings:
    dist = np.linalg.norm(img_emb - query_embedding)
    distances.append(dist)

  sort_index = np.argsort(distances)
  top_k = sort_index[:k]

  retrieved_images = [valid_images[i] for i in top_k]
  correct = label in retrieved_images
  if correct:
    #print("correct")
    correct_output += 1

In [ ]:
accuracy = correct_output/
print('This model was successfully able to retrieve a relevant image', accuracy*100, '% of the time')

This model was successfully able to retrieve a relevant image 68.83116883116884 % of the time


* 20 hidden, num_triplets_per_caption= 3 :

3 epochs - 53.91%, 5 - 66.52% , 6 - 65.21%
* 18 hidden, num_triplets_per_caption= 3 :

3 epochs - 56.08%, 5 - 53.47%
* 20 hidden, num_triplets_per_caption= 5 :

3 epochs - 3 - 56.52%, 5 - 67.82%, 6 - 60%

* 12 hidden, num_triplets_per_caption= 3 :

3 epochs - 49.35%

* 12 hidden, num_triplets_per_caption= 3, LR = 0.0002 :

4 epochs - 59.74% 8 epochs - **68.83%**